In [1]:
import face_recognition
import cv2
import numpy as np
import os
import glob

In [3]:
# Import images
faces_encodings = []
face_names = []
face_locations = []
process_this_frame = True

cur_dir = os.getcwd()
train_path = os.path.join(cur_dir, 'data/train/')
test_path = os.path.join(cur_dir, 'data/test/')
train_file_list = [f for f in glob.glob(train_path + '*.jpeg')]
num_of_files = len(train_file_list)
names = train_file_list.copy()

In [ ]:
# Perform training
for i in range(num_of_files):
    globals()['image_{}'.format(i)] = face_recognition.load_image_file(train_file_list[i])
    globals()['image_encoding_{}'.format(i)] = face_recognition.face_encodings(globals()['image_{}'.format(i)])[0]
    faces_encodings.append(globals()['image_encoding_{}'.format(i)])

    # Create array of known names
    names[i] = names[i].replace(cur_direc, "")  
    face_names.append(names[i])

In [7]:
tr_image = face_recognition.load_image_file(train_path + 'Trinh.jpeg')
unknown_image = face_recognition.load_image_file(test_path + 'IMG_2145.jpeg')
tr_encoding = face_recognition.face_encodings(tr_image)[0]
unknown_encoding = face_recognition.face_encodings(unknown_image)[0]

result = face_recognition.compare_faces([tr_encoding], unknown_encoding)
print(result)

[True]


Reference: <br/>
https://towardsdatascience.com/building-a-face-recognizer-in-python-7fd6630c6340 <br/>
https://github.com/ageitgey/face_recognition